# Import Libraries

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f
from scipy.special import betainc
from scipy.stats import norm, f, t
from scipy.ndimage import rotate
from PIL import Image

# Define Parameters

In [2]:
# Define parameters
subjects = 57

# Define Data

In [3]:
anatomical = nib.load('Unthresholded/neurosynth/anatomical.nii.gz').get_fdata()
spmT = nib.load('Unthresholded/neurosynth/TMap_1.nii.gz').get_fdata()
mask = nib.load('Unthresholded/neurosynth/mask.nii.gz').get_fdata()
spmT_masked=mask*spmT

# SGPV Calculations

In [6]:
# Function to compute z-score from alpha
def z_score_from_alpha(alpha, tail='two-sided'):
    """
    Calculates the z-score from a given alpha value.

    Parameters:
    alpha (float): Significance level (e.g., 0.05).
    tail (str): Type of test tail ('two-sided', 'left', or 'right').
                  Defaults to 'two-sided'.

    Returns:
    float: Z-score corresponding to the alpha value.
    """
    if tail == 'two-sided':
        z = norm.ppf(1 - alpha/2)
    elif tail == 'left':
        z = norm.ppf(alpha)
    elif tail == 'right':
         z = norm.ppf(1 - alpha)
    else:
        raise ValueError("tail must be 'two-sided', 'left', or 'right'")
    return z

In [7]:
# Function for second generation p-value calculations
def calculate_second_gen_p_value(observed_effect, null_hypothesis, effect_interval, n, alpha, df, verbose=True):
    """
    Calculate second-generation p-value for neuroimaging data

    Parameters:
        observed_effect (float): Estimated effect size from neuroimaging analysis - Contrast_img (z scores)
        null_hypothesis (float): Point null hypothesis value (0)
        effect_interval (float): Interval of practically equivalent effects (User provides this - we need to test this value)
        f2 (float): Cohen's f-squared effect size
        n (int): Sample size
        alpha (float): Significance level
        df (int): Degrees of freedom
        verbose (bool): If True, print intermediate calculations. If False, suppress output.

    Returns:
        tuple: (delta_p, interpretation)
    """
    def vprint(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)

    vprint(f'N: {n}')
    vprint(f'alpha: {alpha}')
    vprint(f'df: {df}')

    #vprint(f'Cohens f2: {f2}')
    #d = 2 * np.sqrt(f2) #in the case of 2 means (a t-test) according to Cohen's power book
    #vprint(f'Cohens d: {d}\n')

    vprint(f'Z-score: {observed_effect}')
    #t_val = observed_effect / (d * np.sqrt(n))
    #vprint(f'T-score: {t_val}')
    
    if df == 2:
        std = z_score_from_alpha(alpha, tail='two-sided')
    elif df == 1:
        std = z_score_from_alpha(alpha, tail='right')
    else:
        vprint('error - no df specified')
        std = 1  # fallback to prevent crash
    
    vprint(f'SD: {std}')
    std_error = std / np.sqrt(n - df)
    vprint(f'SE: {std_error}')

    ci_lower = observed_effect - std_error
    ci_upper = observed_effect + std_error
    vprint(f'Confidence Interval: [{ci_lower},{ci_upper}]\n')

    ###TEST
    #effect_interval = effect_interval / (d * np.sqrt(n))
    
    vprint(f'Null Hypothesis: {null_hypothesis}')
    vprint(f'Effect Interval Test Value: {effect_interval}')
    interval_lower = null_hypothesis - effect_interval
    interval_upper = null_hypothesis + effect_interval
    vprint(f'Null Interval: [{interval_lower},{interval_upper}]\n')
    
    if ci_upper <= interval_upper and ci_lower >= interval_lower:
        delta_p = 1.0
        interpretation = 'The data supports the null hypothesis - not scientifically or clinically meaningful'
    elif ci_upper < interval_lower or ci_lower > interval_upper:
        delta_p = 0.0
        interpretation = 'The data supports an alternative hypothesis that is scientifically OR clinically meaningful'
    elif (ci_upper - ci_lower) > (2 * (interval_upper - interval_lower)):
        delta_p = 0.5
        interpretation = 'Data is strictly inconclusive'
    else:
        overlap_lower = max(ci_lower, interval_lower)
        overlap_upper = min(ci_upper, interval_upper)
        delta_p = (overlap_upper - overlap_lower) / (ci_upper - ci_lower)
        interpretation = 'Partial evidence, some consistency with null'

    vprint(f'Second Gen p-value: {delta_p}') 
    vprint(f'Interpretation: {interpretation}')  

    return delta_p

In [16]:
###TEST
calculate_second_gen_p_value(observed_effect = 3.145, #t value
                            null_hypothesis = 0, 
                            effect_interval = 3, #95% CI of t distribution
                            n = subjects, 
                            alpha = 0.05, 
                            df = 1, #1 -tailed
                            verbose=True)

N: 57
alpha: 0.05
df: 1
Z-score: 3.145
SD: 1.644853626951472
SE: 0.219802811551603
Confidence Interval: [2.925197188448397,3.364802811551603]

Null Hypothesis: 0
Effect Interval Test Value: 3
Null Interval: [-3,3]

Second Gen p-value: 0.17015890521045832
Interpretation: Partial evidence, some consistency with null


0.17015890521045832